# Query Intent Classification Hands-on Tutorial

# Introduction

In this session, we'll go through a query/language understanding task in search: query intent classification. 
We will show how to build a production-ready CNN model for query intent classification using the DeText framework. For more details on DeText, a deep neural text understanding framework, please refer to our [github page](https://github.com/linkedin/detext).

## Query intent in search system
Understanding the intents associated with a query accurately allows the search engine to trigger corresponding vertical searches, as well as to better rank the retrieved documents based on the intents.


## Tutorial setup


### Outline
* Environment setup
* Data preprocessing
* Model training
* Inference with trained model


###   Dataset 
We will use the publicly available Natural Language Understanding benchmark dataset ([nlu-benchmark](https://github.com/snipsco/nlu-benchmark/tree/master/2017-06-custom-intent-engines)). The labeled data contains 7 intents including the following: 
* SearchCreativeWork (e.g. Find me the I, Robot television show),
* GetWeather (e.g. Is it windy in Boston, MA right now?),
* BookRestaurant (e.g. I want to book a highly rated restaurant for me and my boyfriend tomorrow night),
* ~~PlayMusic (e.g. Play the last track from Beyoncé off Spotify),~~ This is not included in this tutorial due to dataset download issues.
* AddToPlaylist (e.g. Add Diamonds to my roadtrip playlist)
* RateBook (e.g. Give 6 stars to Of Mice and Men)
* SearchScreeningEvent (e.g. Check the showtimes for Wonder Woman in Paris)

We will use the 6 valid classes to train the query intent classifier.

### Model 
Users will see how to train a CNN model for query intent classification with DeText. The trained model is ready for online inference in production search systems.


### Time required
appox. 20min



# Set up the environment
We first need to install detext using pip.

In [3]:
!pip3 install detext==2.0.8




  Using cached https://files.pythonhosted.org/packages/f2/83/7d78d7663149ba14cd6558a207be934e9d0dd4a9c6f7f9268f4f7ab36fed/detext-2.0.8.tar.gz
     |████████████████████████████████| 105.8MB 2.7MB/s eta 0:00:01   |███                             | 9.9MB 2.2MB/s eta 0:00:43
     |████████████████████████████████| 40kB 2.3MB/s eta 0:00:01
Processing /Users/thchang/Library/Caches/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd/gast-0.2.2-cp37-none-any.whl
     |████████████████████████████████| 3.0MB 3.6MB/s eta 0:00:01
     |████████████████████████████████| 491kB 3.5MB/s eta 0:00:01
Processing /Users/thchang/Library/Caches/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6/termcolor-1.1.0-cp37-none-any.whl
     |████████████████████████████████| 51kB 4.4MB/s eta 0:00:01
     |████████████████████████████████| 133kB 4.0MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/71/e3/19762fdfc62877ae9102edf6342d71b28fbfd9dea3d2f96a882

# Download and preprocess the dataset
In this section, we download the dataset and perform some basic preprocessing to the text data, including lowercasing, whitespace normalization, etc.



In [1]:
!git clone https://github.com/snipsco/nlu-benchmark.git

fatal: destination path 'nlu-benchmark' already exists and is not an empty directory.


In [3]:
%tensorflow_version 1.x
import tensorflow as tf
tf.__version__

UsageError: Line magic function `%tensorflow_version` not found.


In [4]:
# The constants
CLASSES = [
           "AddToPlaylist", 
           "BookRestaurant", 
           "GetWeather", 
           "RateBook", 
           "SearchCreativeWork", 
           "SearchScreeningEvent"
           ]
# See https://github.com/linkedin/detext/blob/master/TRAINING.md for more details on the feature naming format required by DeText
QUERY_FIELDNAME = "doc_query"
WIDE_FTR_FIELDNAME = "wide_ftrs"
LABEL_FIELDNAME = "label"
VOCAB_FILE = "vocab.txt"

In [5]:
import json


train_raw = {}
test_raw = {}

for c in CLASSES:
  with open('/content/nlu-benchmark/2017-06-custom-intent-engines/{}/train_{}_full.json'.format(c, c)) as f:
    train_raw[c] = json.load(f)[c]
  
  with open('/content/nlu-benchmark/2017-06-custom-intent-engines/{}/validate_{}.json'.format(c, c)) as f:
    test_raw[c] = json.load(f)[c]
  
  print("Training samples for {}: {}".format(c, len(train_raw[c])), ", test samples for {}: {}".format(c, len(test_raw[c])))


FileNotFoundError: [Errno 2] No such file or directory: '/content/nlu-benchmark/2017-06-custom-intent-engines/AddToPlaylist/train_AddToPlaylist_full.json'

In [8]:
import re
import string

def proprocess_data(data_raw):
  X = []
  y = []
  for c in CLASSES:
    for sample in data_raw[c]:
      tokens = []
      for d in sample['data']:
        t = d['text'].strip().lower()
        t = re.sub(r'([%s])' % re.escape(string.punctuation), r' \1 ', t) 
        t = re.sub(r'\\.', r' ', t) 
        t = re.sub(r'\s+', r' ', t) 
        tokens.append(t)
      sentence = ' '.join(tokens)

      X.append(sentence)
      y.append(CLASSES.index(c))
  return X, y

    
X_train, y_train = proprocess_data(train_raw)
X_test, y_test = proprocess_data(test_raw)

print(X_train[:5])
print(y_train[:5])

NameError: name 'train_raw' is not defined

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size=0.1, random_state=42)


In [8]:
# Stats
print('Total train samples: {}'.format(len(y_train)))
print('Total dev samples: {}'.format(len(y_dev)))
print('Total test samples: {}'.format(len(y_test)))

Total train samples: 10605
Total dev samples: 1179
Total test samples: 600


# Prepare DeText dataset

Now that we have preprocessed the datasets and prepared the train/ dev/ test splits, let's convert the data samples into the correct format that DeText accepts. 

The text inputs are converted to `byte_list` features and labels are converted to `float_list` features. The datasets are store in `tfrecord` format. For more information on the input format of DeText, please checkout [the training manual](https://github.com/linkedin/detext/blob/master/TRAINING.md).

In [9]:
import tensorflow as tf
 
def create_float_feature(value): 
    feature = tf.train.Feature(float_list=tf.train.FloatList(value=[value])) 
    return feature 
 
 
def create_byte_feature(value): 
    feature = tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
    return feature 
 

def create_tfrecords(X, y, output_file):
  assert len(X) == len(y)
  total_records = 0 
  writer = tf.python_io.TFRecordWriter(output_file) 
  for q, c in zip(X, y):
    features = {} 
    features[QUERY_FIELDNAME] = create_byte_feature(str.encode(q))
    features[LABEL_FIELDNAME] = create_float_feature(c)
    features[WIDE_FTR_FIELDNAME] = create_float_feature(len(q.split()))
    tf_example = tf.train.Example(features=tf.train.Features(feature=features)) 
    if total_records < 2: 
      print(tf_example) 
    writer.write(tf_example.SerializeToString()) 
    total_records += 1 
  print("processed {} records".format(total_records)) 

create_tfrecords(X_train, y_train, 'train.tfrecord')
create_tfrecords(X_dev, y_dev, 'dev.tfrecord')
create_tfrecords(X_test, y_test, 'test.tfrecord')

features {
  feature {
    key: "doc_query"
    value {
      bytes_list {
        value: "i want to bring antoinette , christine and caitlin to get bread at a restaurant in faraway  connecticut"
      }
    }
  }
  feature {
    key: "label"
    value {
      float_list {
        value: 1.0
      }
    }
  }
  feature {
    key: "wide_ftrs"
    value {
      float_list {
        value: 18.0
      }
    }
  }
}

features {
  feature {
    key: "doc_query"
    value {
      bytes_list {
        value: "book a reservation for 4 at a bar serving corn relish"
      }
    }
  }
  feature {
    key: "label"
    value {
      float_list {
        value: 1.0
      }
    }
  }
  feature {
    key: "wide_ftrs"
    value {
      float_list {
        value: 11.0
      }
    }
  }
}

processed 10605 records
features {
  feature {
    key: "doc_query"
    value {
      bytes_list {
        value: "book a restaurant in everson  nevada that serves conchiglie for two people . "
      }
    }
  }
  feat

In [7]:
# Generate vocab
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X_train)
vocab = ['[PAD]', '[UNK]']
vocab.extend(list(vectorizer.vocabulary_.keys()))
print(vocab)

# Write vocab to file
with open(VOCAB_FILE, 'w') as f:
  for v in vocab:
    f.write("{}\n".format(v))


NameError: name 'X_train' is not defined

# DeText training
We will train a multi-class classification model using the DeText framework. 



In [5]:
# See https://github.com/linkedin/detext/blob/master/TRAINING.md for more details on the parameters

from detext.run_detext import run_detext,DetextArg

args = DetextArg(num_classes=6,
                ftr_ext="cnn",
                num_filters=50,
                num_units=64,
                num_wide=1,
                optimizer="bert_adam", # same AdamWeightDecay optimizer as in BERT training
                learning_rate=0.001,
                max_len=16,
                min_len=3,
                use_deep=True,
                num_train_steps=300,
                steps_per_stats=1,
                steps_per_eval=30,
                train_batch_size=64,
                test_batch_size=64,
                pmetric="accuracy",
                all_metrics=["accuracy", "confusion_matrix"],
                vocab_file=VOCAB_FILE,
                feature_names=["label","doc_query","wide_ftrs"],
                train_file="train.tfrecord",
                dev_file="dev.tfrecord",
                test_file="test.tfrecord",
                out_dir="output")
run_detext(args)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





NameError: name 'VOCAB_FILE' is not defined

# Let's check the predictions

In [12]:
import glob
from tensorflow.contrib import predictor 
saved_model_path = glob.glob('output/export/best_accuracy/*')[0]

print(saved_model_path)

INFO:tensorflow:output/export/best_accuracy/1598311673


In [13]:
!saved_model_cli show --dir output/export/best_accuracy/* --tag_set serve --signature_def serving_default

INFO:tensorflow:The given SavedModel SignatureDef contains the following input(s):
  inputs['doc_query'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: doc_query_placeholder:0
  inputs['label'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1)
      name: label_placeholder:0
  inputs['uid'] tensor_info:
      dtype: DT_INT64
      shape: ()
      name: uid_placeholder:0
  inputs['weight'] tensor_info:
      dtype: DT_FLOAT
      shape: ()
      name: weight_placeholder:0
  inputs['wide_ftrs'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: wide_ftr_placeholder:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['label'] tensor_info:
      dtype: DT_FLOAT
      shape: (1, -1)
      name: ExpandDims_2:0
  outputs['multiclass_probabilities'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 6)
      name: Softmax:0
  outputs['scores'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 6)
      name: Squeeze:0
  out

In [14]:
import numpy as np
predict_fn = predictor.from_saved_model(saved_model_path) 

def predict_class(query):
  predictions = predict_fn( 
                  {"doc_query": [query], 
                   "wide_ftrs": [[len(query.split())]],
                   # dummy pass-through input for label field -- this is not used in making predictions
                   "label": [1.0]}) 
  probabilities = predictions['multiclass_probabilities'][0]
  predicted_class = CLASSES[np.argmax(probabilities)]
  print("query \"{}\", predicted intent \"{}\".".format(query, predicted_class))

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from output/export/best_accuracy/1598311673/variables/variables


In [15]:
# GetWeather intent
predict_class("how's the weather in Mountain View")

# BookRestaurant intent
predict_class("I need a table for 4")

# PlayMusic intent
predict_class("put Jean Philippe Goncalves onto my running to rock 170 to 190 bpm")

# SearchScreeningEvent intent
predict_class("Find the schedule for The Comedian")

INFO:tensorflow:query "how's the weather in Mountain View", predicted intent "GetWeather".
INFO:tensorflow:query "I need a table for 4", predicted intent "BookRestaurant".
INFO:tensorflow:query "put Jean Philippe Goncalves onto my running to rock 170 to 190 bpm", predicted intent "AddToPlaylist".
INFO:tensorflow:query "Find the schedule for The Comedian", predicted intent "SearchScreeningEvent".


# References

\[1\] Dataset: Snips Voice Platform: an embedded Spoken Language Understanding system for private-by-design voice interfaces, [github repo](https://github.com/snipsco/nlu-benchmark)

\[2\] DeText: A Deep Neural Text Understanding Framework [github repo](https://github.com/linkedin/detext)

